In [8]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [104]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [119]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

# Sampling 6h and 10h were only used for sec not NMR -> no conversion for these.
skip_hours_mask = [True if h not in hours_list else False for h in ext_time_corr_df.columns]
ext_time_corr_df.drop(columns=time_cols[skip_hours_mask], inplace=True)

ext_time_corr_df



The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,8,15
Reactor,,,,,,
1,0,60.333333,120.666667,241.083333,481.000000,901.000000
2,0,60.333333,120.666667,241.083333,481.000000,901.000000
3,0,64.416667,124.083333,244.666667,485.416667,905.916667
4,0,64.416667,124.083333,244.666667,485.416667,905.916667
5,0,67.916667,127.500000,248.583333,490.833333,912.000000
6,0,67.916667,127.500000,248.583333,490.833333,912.000000
7,0,71.500000,131.083333,252.333333,495.416667,917.500000
8,0,71.500000,131.083333,252.333333,495.416667,917.500000
9,0,75.083333,134.500000,256.416667,500.333333,922.916667


In [122]:
# srt.df.columns
ext_time_corr_df.loc["15"]


0       0.000000
1      86.250000
2     145.833333
4     268.500000
8     514.916667
15    939.500000
Name: 15, dtype: float64

In [138]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame(
        {"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list, "reactor" : polymerisation_kinetic["reactor"]})
    kinetic_curve_entries["experiment"] = polymerisation_kinetic["Experiment number"]+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [139]:
kinetic_curves_df
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

,conversion,time,reactor,experiment
t0h-conversion,0,0.000000,1,MRG-046-G_6_A
t1h-conversion,0.086971,60.333333,1,MRG-046-G_6_A
t2h-conversion,0.104365,120.666667,1,MRG-046-G_6_A
t4h-conversion,0.205776,241.083333,1,MRG-046-G_6_A
t8h-conversion,0.33213,481.000000,1,MRG-046-G_6_A
...,...,...,...,...
t1h-conversion,0.298969,82.416667,14,MRG-046-G_10_E
t2h-conversion,0.436426,142.166667,14,MRG-046-G_10_E
t4h-conversion,1.0,264.583333,14,MRG-046-G_10_E
t8h-conversion,1.0,510.083333,14,MRG-046-G_10_E


In [159]:
# and fit curve and the derivation
# find the maximum of the derivation

'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

In [169]:
fig = px.line(title="kinetic curve fit", labels={"x":"time", "y":"conversion"})

# set up a simple color palette

for idx, kinetic_curve in enumerate(kinetic_curves):
    # choosing a few test function
    if not idx in (1, 2, 3):
    # if not idx in (39, 40):
        continue

    ydata = kinetic_curve["conversion"].values.astype(float)
    xdata = kinetic_curve["time"].values

    p0 = [max(ydata), 1] # this is a mandatory initial guess
    popt, pcov = curve_fit(f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10)
    print(idx)

    # plot the fit and data
    extended_xdata = np.linspace(min(xdata)-2, max(xdata)+2, 100)
    y_extended = neg_growth(extended_xdata, *popt)
    notnan_extended = ~(np.isnan(extended_xdata) | np.isnan(y_extended))
    extended_xdata = extended_xdata[notnan_extended]
    y_extended = y_extended[notnan_extended]
#    extended_xdata = np.array(list(range(min(xdata)-2, min(xdata)-1, 1)) + xdata + list(range(max(xdata)+1, max(xdata)+3, 1)))
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1)
    fig.add_scatter(x=extended_xdata, y=neg_growth(extended_xdata, *popt), opacity=1, line=dict(dash="dot"))
    fig.add_scatter(x=extended_xdata, y=neg_growth_derivative(extended_xdata, *popt), opacity=0.3, line=dict(dash="dot"))
    # print(neg_growth(extended_xdata, *popt))
    # break
fig.update_layout(
    yaxis=dict(
        range=[-0.5,1]
    ),
    xaxis_title="time",
    yaxis_title="conversion"
)
fig.show()



1
2
3


C:\Users\Nex\mambaforge\envs\PFP2\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



In [141]:
# get the maximum of the derivative
print(len(kinetic_curves))

330


In [155]:
ttpm1= 1, 1
ttpm2= 2, 1
ttpm4= 1, 2

xrange = np.arange(-0.5, 0.5, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm4):
    newfig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams), mode="lines", opacity=1)
#    newfig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5)

newfig.show()

In [158]:
ttpm1= 1, 1
ttpm2= 2, 1
ttpm3= 1, 2

xrange = np.arange(-0.5, 5, 0.1)
newfig = px.line()
for testparams in (ttpm1, ttpm2, ttpm3):
    newfig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    newfig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

newfig.show()